In [100]:
import pandas as pd
import os
from collections import Counter

In [90]:
# read in 
df = pd.read_csv('../raw_dssp_csv/1i3n.csv')
# replace 'NaN' with '?'
df.fillna(value='?', inplace=True)
# show
df.head(10)

,DSSP,PDB,CHAIN,AA,SS,3H,4H,5H,BEND,CHIR,...,ON2I,ON2E,TCO,KAPPA,ALPHA,PHI,PSI,X-CA,Y-CA,Z-CA
0,1,2,A,A,?,?,?,?,?,?,...,79,-0.2,0,360,360,360,-1.4,-4.8,7.2,22.5
1,2,3,A,E,?,?,?,?,?,-,...,79,-0.2,-0.681,360,-126.5,-88.8,145.6,-1.4,8.7,21.7
2,3,4,A,K,E,?,?,?,?,-,...,80,-0.9,-0.3,2.4,-131.5,-88.8,174.4,1.3,7.7,24.2
3,4,5,A,V,E,?,?,?,?,-,...,2,-0.4,-0.993,20.7,-141.5,-123.8,122.5,3.8,9.6,26.3
4,5,6,A,L,E,?,?,?,?,-,...,2,-0.5,-0.735,14.2,-164.6,-85.6,131.1,7.4,8.4,26.1
5,6,7,A,V,E,>,?,?,?,-,...,3,-0.8,-0.945,3.6,-159.8,-118.7,106.8,9.2,8.5,29.5
6,7,8,A,T,E,3,?,?,S,+,...,25,-0.2,-0.651,79.5,12.3,-85,145.9,13,8.2,29.3
7,8,9,A,G,T,>,?,?,S,+,...,6,-0.4,0.792,83.3,153.9,62.7,25.1,14.9,7.1,32.4
8,9,10,A,G,T,<,?,?,?,+,...,23,-0.1,0.541,56.7,65.3,-66.2,-9.4,11.4,6.1,33.8
9,10,11,A,A,T,3,?,?,S,+,...,28,-0.2,0.578,88.6,88.7,-89.4,-9.4,12.8,3.4,36.1


In [91]:
df.shape

(694, 32)

In [92]:
# drop '!*' rows
# df = df.drop(df[df['AA'] == '!'].index.values)

df = df[df['AA'] != '!']

In [93]:
df.shape

(693, 32)

In [94]:
# if row was successfully dropped
try:
    df.ix[347]
except: 
    print 'success'

success


In [95]:
# fix HHTHH structure pattern
def replace_pi(l):
    i = 0
    
    while i < len(l)-4:
        window = ''.join(l[i:i+5])
        
        if window == 'HHTHH':
            l[i:i+5] = 'I'
    
        i += 1
        
    return l

In [96]:
# replace the "HHTHH" pattern with "IIIII"
df['SS'] = replace_pi(df['SS'])

In [97]:
# replace the '?' with 'C'
df['SS'] = df['SS'].apply(lambda aa: 'C' if aa=='?' else aa)

In [98]:
df.head(10)

,DSSP,PDB,CHAIN,AA,SS,3H,4H,5H,BEND,CHIR,...,ON2I,ON2E,TCO,KAPPA,ALPHA,PHI,PSI,X-CA,Y-CA,Z-CA
0,1,2,A,A,C,?,?,?,?,?,...,79,-0.2,0,360,360,360,-1.4,-4.8,7.2,22.5
1,2,3,A,E,C,?,?,?,?,-,...,79,-0.2,-0.681,360,-126.5,-88.8,145.6,-1.4,8.7,21.7
2,3,4,A,K,E,?,?,?,?,-,...,80,-0.9,-0.3,2.4,-131.5,-88.8,174.4,1.3,7.7,24.2
3,4,5,A,V,E,?,?,?,?,-,...,2,-0.4,-0.993,20.7,-141.5,-123.8,122.5,3.8,9.6,26.3
4,5,6,A,L,E,?,?,?,?,-,...,2,-0.5,-0.735,14.2,-164.6,-85.6,131.1,7.4,8.4,26.1
5,6,7,A,V,E,>,?,?,?,-,...,3,-0.8,-0.945,3.6,-159.8,-118.7,106.8,9.2,8.5,29.5
6,7,8,A,T,E,3,?,?,S,+,...,25,-0.2,-0.651,79.5,12.3,-85,145.9,13,8.2,29.3
7,8,9,A,G,T,>,?,?,S,+,...,6,-0.4,0.792,83.3,153.9,62.7,25.1,14.9,7.1,32.4
8,9,10,A,G,T,<,?,?,?,+,...,23,-0.1,0.541,56.7,65.3,-66.2,-9.4,11.4,6.1,33.8
9,10,11,A,A,T,3,?,?,S,+,...,28,-0.2,0.578,88.6,88.7,-89.4,-9.4,12.8,3.4,36.1


In [101]:
Counter(df['SS'].values)

Counter({'B': 8,
         'C': 153,
         'E': 120,
         'G': 11,
         'H': 254,
         'I': 5,
         'S': 81,
         'T': 61})

## Now for all of the dssp_csv files

In [103]:
from __future__ import print_function

In [73]:
topdir, _, files = next(os.walk('../raw_dssp_csv/'))

In [104]:
for i, fi in enumerate(files):
    # show progress
    print('\r On protein #'+str(i), end='')
    
    # read in the csv
    df = pd.read_csv(topdir+fi)
    
    # replace NaN with ?
    df.fillna(value='?', inplace=True)
    
    # drop !* rows
    df = df[df['AA'] != '!']
    
    # replace "HHTHH" pi helix pattern 
    df['SS'] = replace_pi(df['SS'])
    
    # replace '?' with 'C'
    df['SS'] = df['SS'].apply(lambda aa: 'C' if aa == '?' else aa)
    
    # write to csv
    df.to_csv('../clean_dssp_csv/'+fi, index=False)

 On protein #346